In [1]:
from sliderule import icesat2
import earthaccess as ea
import time
import numpy as np
from datetime import datetime
import os
from matplotlib import pyplot as plt

### One Granule

In [31]:
def get_size(gdf):
    return gdf.memory_usage(deep=True).sum() / (1024**2)

In [ ]:
granule_path = 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2019/12/02/ATL06_20191202203649_10220511_006_01.h5'
granule = os.path.basename(granule_path)

dt = datetime.strptime(granule.split('_')[1][0:8], "%Y%m%d")
dt_str = datetime.strftime(dt, "%Y-%m-%dT%H:%M:%SZ")


#### One granule, One beam, Subsetted

In [60]:
parms_bbox = {
    "poly": [
    {
    "lon": 152.84367053387408,
    "lat": -81.2185940279265
    },
    {
    "lon": 157.30604049933277,
    "lat": -80.7324891420691
    },
    {
    "lon": 158.0479504202403,
    "lat": -80.55684138323734
    },
    {
    "lon": 157.09874213907918,
    "lat": -80.39247953410985
    },
    {
    "lon": 151.34894025204565,
    "lat": -80.9204126913049
    },
    {
    "lon": 152.84367053387408,
    "lat": -81.2185940279265
    }
    ],
    "beams": 'gt2l',
    }

In [ ]:
tic = time.time()
D6 = icesat2.atl06s(parm=parms_bbox, 
  resource='ATL06_20191202203649_10220511_006_01.h5')
toc = time.time()
print(toc-tic)
print(f'{get_size(D6)} MB')

2.1990058422088623


np.float64(0.004886627197265625)

##### One Granule, One beam, No subsetting

In [54]:
tic = time.time()
D6 = icesat2.atl06sp(parm={
  "beams": 'gt2l',
  }, 
  resources=['ATL06_20191202203649_10220511_006_01.h5'])
toc = time.time()
print(toc-tic)
print(f'{get_size(D6)} MB')

13.770008087158203
9.040180206298828 MB


### 9 granules

In [62]:
# bigger bounding box (greenland) (don use these thos)

parms_bbox= {
  "poly" : [
    {"lon": -45.4, "lat": 62.63},    # lower left
    {"lon": -44.6, "lat": 62.63},    # lower right
    {"lon": -44.6, "lat": 63.0},     # upper right
    {"lon": -45.4, "lat": 63.0},     # upper left
    {"lon": -45.4, "lat": 62.63}     # close polygon
  ],
  "cycle": 25,
}

#### 9 Granules, Subsetted

In [59]:
## 

tic = time.time()
D6_bbox = icesat2.atl06sp(parm=parms_bbox)
toc = time.time()
print(toc-tic)
print(f'{get_size(D6_bbox)} MB')

49.926422119140625
100.09502792358398 MB


#### 9 Granules, No subsetting

In [49]:
results = ea.search_data(
    short_name='ATL06',
    cloud_hosted=True,
    bounding_box=(-45.4, 62.63, -44.6, 63.0),   
    granule_name="ATL06_*_????25??_*_*.h5",
)
granules = [os.path.basename(g.data_links()[0]) for g in results]


In [ ]:
tic = time.time()
D6 = icesat2.atl06sp(parm={}, resources=granules)
toc = time.time()
print(toc-tic)
print(f'{get_size(D6)} MB')

185.89279532432556


np.float64(363.0933837890625)

#### 9 granules, no subsetting

In [ ]:
tic = time.time()
D6_resources_bbox = icesat2.atl06sp(parm=parms_bbox, resources=granules)
toc = time.time()
print(toc-tic)
print(f'{get_size(D6_resources_bbox)} MB')

3.640393018722534
9.040180206298828 MB


In [63]:
tic = time.time()
D6_resources_bbox = icesat2.atl06sp(parm=parms_bbox)
toc = time.time()
print(toc-tic)
print(f'{get_size(D6_resources_bbox)} MB')

15.738665103912354
5.994529724121094 MB


### Results

| Granules | Function | Method | Speed (s) | Size (MB) | Notes           |
|----------|----------|---------|-----------|-----------|-----------|
| 1 |   s  |  granule |  5.2 |  9.04  | gt2l  |
| 1 |   s  |  bbox |  3.0 |  0.005  | gt2l    |
| 1 |  sp  |  granule | 5.3 | 9.04  |  gt2l  |
| 1 |  sp  |  bbox |  2.6 | 0.005  |  gt2l  |
| 9 |   sp  |  granule |  4 m 12 s |    363.1    | cycle 25 |
| 9 |  sp  |  bbox | 8.7  | 6.0  |  cycle 25  |
| 9 |  sp  |  bbox | 49.3  | 100.1  |  all  |